# Lasso Scenario Creation Quickstart

In this notebook we will run through:

1. Using a configuration file to run lasso  
2. Setting up a base sc
enario and applying projects  
3. Transforming the standard network format to the MetCouncil expected format    
4. Exporting the network to a shapefile and csvs 

In [1]:
import os
import sys
import yaml

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.DEBUG)


## 1 - Set Base Roadway and Transit Network Locations

In [4]:
# Examples set here for testing purposes
EXAMPLE_ROADWAY_DIRECTORY = os.path.join(
    os.path.dirname(os.path.abspath('')), "examples", "stpaul"
)

EXAMPLE_TRANSIT_DIRECTORY = os.path.join(
    os.path.dirname(os.path.abspath('')), "examples", "stpaul"
)

In [5]:
# set this to the path where your roadway files are  i.e. r"c:\networks\myroad\base"
BASE_ROADWAY_DIRECTORY = EXAMPLE_ROADWAY_DIRECTORY

# set this to the path where your transit files are i.e. r"c:\networks\mytransit\base"
BASE_TRANSIT_DIRECTORY = EXAMPLE_TRANSIT_DIRECTORY

## 2 - Scenario Settings

In [6]:
# Examples set here for testing purposes
EXAMPLE_PROJECT_CARD_DIRECTORY = os.path.join(
    os.path.dirname(os.path.abspath('')), "examples", "stpaul", "project_cards"
)

EXAMPLE_OUTPUT_DIRECTORY = os.path.join(
    os.path.dirname(os.path.abspath('')), "tests", "scratch"
)

In [7]:
# set this to the path of the folder where the project cards are i.e. r"C:\networks\project_cards"
PROJECT_CARD_DIRECTORY = EXAMPLE_PROJECT_CARD_DIRECTORY

# set this to the path of the folder you want output to b e i.e. r"C:\networks\built_nets\base"
OUTPUT_DIRECTORY = EXAMPLE_OUTPUT_DIRECTORY

## 3 - Create Scenario

### Base Scenario: what networks are you adding the projects to?

Base scenarios must at the least specify a highway network but can also specify a directory where transit networks can be found.  

In this step the highway and transit networks are read in and validated to each other – making sure that transit nodes exist and the routes can be followed along the roadway network..  

Validation can take some time, so if you are already confident that your roadway and transit networks are valid and compatible you may want to override the validation using the flag: `validate = False`.

In [8]:
from network_wrangler import Scenario

In [9]:
base_scenario = Scenario.create_base_scenario(
        "shape.geojson",
        "link.json",
        "node.geojson",
        roadway_dir=BASE_ROADWAY_DIRECTORY,
        transit_dir=BASE_TRANSIT_DIRECTORY,
        validate=False,
        shape_foreign_key="shape_id",
    )

2021-04-07 09:02:42, INFO: Reading RoadwayNetwork
2021-04-07 09:02:51, INFO: Read 66253 links from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/Lasso/examples/stpaul/link.json
2021-04-07 09:02:51, INFO: Read 17159 nodes from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/Lasso/examples/stpaul/node.geojson
2021-04-07 09:02:51, INFO: Read 66253 shapes from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/Lasso/examples/stpaul/shape.geojson
2021-04-07 09:03:07, INFO: Adding transit feed from: /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/Lasso/examples/stpaul
2021-04-07 09:03:07, INFO: Removing calendar.txt from transit network config because file not found
2021-04-07 09:03:07, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-04-07 09:03:07, INFO: Removing fare_attributes.txt from transit network config because file not found
2021-04-07 09:03:07, INFO: Removing fare_rules.txt from transit network config because 

### Scenario Creation: adding projects to the base scenario

As coded, this will add projects from the `PROJECT_CARD_DIRECTORY` and if specified here, will filter based on a glob search or a tag.

In [10]:
my_fun_scenario=None

my_fun_scenario = Scenario.create_scenario(
    base_scenario=base_scenario,
    card_directory=PROJECT_CARD_DIRECTORY,
    #tags=,   #uncomment if you want to use
    #glob_search=,  #uncomment if you want to use
    validate_project_cards=False,
)

2021-04-07 09:03:20, INFO: Creating Scenario


In [11]:
my_fun_scenario.apply_all_projects()

2021-04-07 09:03:27, INFO: Applying Bus Reroute
2021-04-07 09:03:27, INFO: Applying Project to Transit Network: Bus Reroute
2021-04-07 09:03:27, INFO: Applying Blue Line Service Enhancement A
2021-04-07 09:03:27, INFO: Applying Project to Transit Network: Blue Line Service Enhancement A
2021-04-07 09:03:27, INFO: Applying 6th Street Transitway
2021-04-07 09:03:27, INFO: Applying Project to Roadway Network: 6th Street Transitway


/Users/elizabeth/anaconda3/envs/wrangler/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/elizabeth/anaconda3/envs/wrangler/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


### Standard Networks --> Model Networks

At this point, we need to calculate all the variables into what MetCouncil's model is expecting. The method `roadway_standard_to_met_council_network()` broadly does the following:  
 
- creates a parallel managed lane network
- calculates additional variables based on geography or other variables (i.e. county, assignment group, area type, etc)
- flattens variables stored as continuous time values and determines their value by time period (i.e. lanes_am)   
- reprojects into MetCouncil's projection

In [12]:
from lasso import Parameters

2021-04-07 09:03:27, INFO: Lasso base directory set as: /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/Lasso
2021-04-07 09:03:27, INFO: Lasso base directory set as: /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/Lasso


In [13]:
from lasso.metcouncil.defaults import _MC_DEFAULT_PARAMS

my_parameters = Parameters.initialize(base_params_dict = _MC_DEFAULT_PARAMS)

2021-04-07 09:04:04, WARNING: [Parameters.__post_init__()] The following parameters were not used.
                        Check spelling.
['network_to_demand_time_periods', 'roadway_output_espg', 'lookups', 'roadway_value_lookups', 'time_period_abbr_to_names', 'transit_class', 'output_fields', 'transit_to_network_time_periods', 'category_grouping', 'counts', 'time_period_vol_split', 'properties_to_split_by_category_groupings', 'roadway_overlays', 'properties_to_split_by_network_time_periods', 'count_tod_split_fields', 'roadway_field_mappings', 'time_period_abbr_to_time', 'field_type', 'transit_value_lookups']


In [14]:
from lasso.metcouncil import MetCouncilRoadwayNetwork

In [15]:
model_road_net = MetCouncilRoadwayNetwork.from_RoadwayNetwork(
    my_fun_scenario.road_net, parameters=my_parameters,
)
model_road_net.roadway_standard_to_met_council_network()

2021-04-07 09:04:05, INFO: Converting RoadwayNetwork to <class 'lasso.metcouncil.metcouncil_roadway.MetCouncilRoadwayNetwork'> flavor.
2021-04-07 09:04:05, WARNING: Update called but nothing to update. Given: {}
2021-04-07 09:04:05, INFO: Transforming the standard roadway network into the format and variables that             metcouncil's model is expecting
2021-04-07 09:04:05, INFO: Max number of managed lanes is zero (ML_lanes    0
dtype: int64).
2021-04-07 09:04:12, INFO: Creating metcouncil calculated roadway variables.
2021-04-07 09:04:12, INFO: LINKS_DF.CRS: 4326


/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/Lasso/lasso/model_roadway.py:540: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  _link_centroids_gdf["geometry"] = _link_centroids_gdf["geometry"].centroid


2021-04-07 09:04:22, INFO: LINKS_DF.CRS: 4326


/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/Lasso/lasso/model_roadway.py:540: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  _link_centroids_gdf["geometry"] = _link_centroids_gdf["geometry"].centroid


2021-04-07 09:04:29, INFO: LINKS_DF.CRS: 4326


/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/Lasso/lasso/model_roadway.py:540: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  _link_centroids_gdf["geometry"] = _link_centroids_gdf["geometry"].centroid


2021-04-07 09:04:39, INFO: Adding Counts
2021-04-07 09:04:48, WARNING: couldn't coerce column centroidconnect to a <class 'int'>.
2021-04-07 09:04:48, WARNING: couldn't coerce column centroidconnect to a <class 'int'>.
2021-04-07 09:04:48, WARNING: couldn't coerce column count_daily to a <class 'int'>.
2021-04-07 09:04:48, WARNING: couldn't coerce column count_AM to a <class 'int'>.
2021-04-07 09:04:48, WARNING: couldn't coerce column count_MD to a <class 'int'>.
2021-04-07 09:04:48, WARNING: couldn't coerce column count_PM to a <class 'int'>.
2021-04-07 09:04:48, WARNING: couldn't coerce column count_NT to a <class 'int'>.
2021-04-07 09:04:50, INFO: Setting Coordinate Reference System to EPSG 26915


In [16]:
from lasso.cube import CubeTransit

In [17]:
model_transit = CubeTransit.from_source(my_fun_scenario.transit_net)

2021-04-07 09:04:54, WARNING: Update called but nothing to update. Given: {}


## 4 -  Export Networks

### Roadway Model Network --> Shapefiles

In [18]:
model_road_net.write_roadway_as_shp(
    model_road_net.model_links_df,
    model_road_net.model_nodes_df,
    output_directory = ".",
    output_prefix = "my_net",
)

2021-04-07 09:04:54, INFO: Writing Network to shapefiles: 
   -links ./my_netlinks_out.shp
   -lnodes ./my_netnodes_out.shp
2021-04-07 09:04:54, INFO: Renaming variables so that they are DBF-safe


AttributeError: 'dict' object has no attribute 'net_to_dbf'

###  Roadway Model Network --> Fixed Width for Cube

In [ ]:
model_road_net.write_roadway_as_fixedwidth(
    model_road_net.model_links_df,
    model_road_net.model_nodes_df,
    output_directory = ".",
    output_prefix = "my_net_ff",
)

### Transit Networks --> Cube Lin File

In [ ]:
model_transit.write_cube(outpath="t_transit_test_from_cube.lin")